# 02 Baseline
## Library: Keras
## Dataset: Features02

In [ ]:
import config
from utils import data_model
from utils.keras import losses
from utils.features02_dlib import FEATURES, TARGETS

import pandas as pd
import numpy as np
from tensorflow.contrib import keras

## Load data

In [ ]:
data, imgs_left, imgs_right = data_model.load(
    config.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_CSV,
    config.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_IMGS_LEFT,
    config.PATH_DATA_FEATURES02_DLIB_AUGMENTED_NORM_IMGS_RIGHT
)

## Split data

In [ ]:
(
    (train_data, train_imgs_left, train_imgs_right),
    (validation_data, validation_imgs_left, validation_imgs_right),
    (test_data, test_imgs_left, test_imgs_right)
) = data_model.split(
    data, imgs_left, imgs_right,
    train_size=0.95,
    validation_size=0.95,
    random_state=42
)

In [ ]:
print("Train length: {}".format(len(train_data)))
print("Validation length: {}".format(len(validation_data)))
print("Test length: {}".format(len(test_data)))

## Model

### Architecture

In [ ]:
img_width, img_height = config.FEATURES02_EYES_SIZE
img_shape = (img_height, img_width)

def get_model():

    # Inputs
    left_imgs = keras.layers.Input(shape=img_shape, name='left_imgs', dtype='float32')
    right_imgs = keras.layers.Input(shape=img_shape, name='right_imgs', dtype='float32')
    features = keras.layers.Input(shape=(len(FEATURES),), name='features', dtype='float32')
    # Left imgs
    l1 = keras.layers.Dense(512, activation=keras.activations.relu)(keras.layers.Flatten()(left_imgs))
    l2 = keras.layers.Dense(256, activation=keras.activations.relu)(l1)
    l3 = keras.layers.Dense(128, activation=keras.activations.relu)(l2)
    ld = keras.layers.Dropout(DROPOUT)(l3)
    # Right imgs
    r1 = keras.layers.Dense(512, activation=keras.activations.relu)(keras.layers.Flatten()(right_imgs))
    r2 = keras.layers.Dense(256, activation=keras.activations.relu)(r1)
    r3 = keras.layers.Dense(128, activation=keras.activations.relu)(r2)
    rd = keras.layers.Dropout(DROPOUT)(r3)
    # Features
    f1 = keras.layers.Dense(128, activation=keras.activations.relu)(features)
    f2 = keras.layers.Dense(64, activation=keras.activations.relu)(f1)
    # Concatenate
    concat = keras.layers.Concatenate()([ld,rd,f2])
    # Dense
    d_1 = keras.layers.Dense(512, activation=keras.activations.relu)(concat)
    d_2 = keras.layers.Dense(128, activation=keras.activations.relu)(d_1)
    d_3 = keras.layers.Dense(32, activation=keras.activations.relu)(d_2)
    d_4 = keras.layers.Dense(2)(d_3)
    # Model
    model = keras.models.Model(
        inputs=[left_imgs, right_imgs, features],
        outputs=[d_4]
    )
    return model

### Parameters

In [ ]:
MODEL_NAME = 'f02_03-06'

EPOCHS = 300
BATCH_SIZE = 128
LEARNING_RATE = 0.005
DECAY = 0.0001
DROPOUT = 0.01

LOSS = losses.mean_euclidean

In [ ]:
model = get_model()


model.compile(
    loss=LOSS,
    metrics=[losses.mean_euclidean],
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=DECAY)
)

print("Parameters to adjust: {}".format(
    np.sum([keras.backend.count_params(p) for p in set(model.trainable_weights)])
))

### Train

In [ ]:
model.fit(
    x={
        'left_imgs':train_imgs_left,
        'right_imgs': train_imgs_right,
        'features': train_data[FEATURES].as_matrix()
    },
    y=train_data[TARGETS].as_matrix(),
    validation_data=(
        {
            'left_imgs': validation_imgs_left,
            'right_imgs': validation_imgs_right,
            'features': validation_data[FEATURES].as_matrix()
        },
        validation_data[TARGETS].as_matrix()
    ),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1, callbacks=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0
)

In [ ]:
model.save(config.PATH_MODELS_KERAS+MODEL_NAME)

### Test

In [ ]:
model_test = keras.models.load_model(
    filepath=config.PATH_MODELS_KERAS+MODEL_NAME,
    custom_objects={
        "mean_euclidean": losses.mean_euclidean,
        "ms_euclidean": losses.ms_euclidean,
        "reg_mean_euclidean": losses.reg_mean_euclidean
    }
)

In [ ]:
model.evaluate(
    x={
        'left_imgs':test_imgs_left,
        'right_imgs': test_imgs_right,
        'features': test_data[FEATURES].as_matrix()
    },
    y=test_data[TARGETS].as_matrix(),
    batch_size=1,
    verbose=1, sample_weight=None
)

In [ ]:
0.12337
0.1164
0.1784
0.179

[0.2070697691578548, 0.2070697691578548]
loss: 0.0319 - mean_euclidean: 0.0319 - val_loss: 0.1428 - val_mean_euclidean: 0.1428
                

128*64*2  +16 > 256*64*31*2
                
68 pts
loss: 0.0338 - mean_euclidean: 0.0338 - val_loss: 0.1626 - val_mean_euclidean: 0.1626
0.1661774875152163

facepoints
0.0368 - mean_euclidean: 0.0368 - val_loss: 0.1659 - val_mean_euclidean: 0.1659
0.17


68
mean_euclidean: 0.0196 - val_loss: 0.1443 - val_mean_euclidean: 0.1443
0.14271

tanh

### Results

Corrected dataset


| Name | Epochs | Batch Size | Learning rate | Decay | Loss | Train | Validation | Test |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| baseline-08a2 | 150 | 128| 0.0005 | 0.00001 | mean_euclidean | 0.0306 | 0.1286 | 0.1245 |
| f02_baseline-09 | 150 | 128| 0.0005 | 0.00001 | mean_euclidean | 0.0384 | 0.1205 | 0.1182 |
| f02_baseline-10 | 150 | 128| 0.0005 | 0.00001 | ms_euclidean | 0.0016 | 0.0234 | 0.02287 |
| f02_baseline-13 | 150 | 128| 0.0005 | 0.00001 | mean_euclidean | 0.0381 | 0.1197 | 0.1199 |
| f02_baseline-14 | 150 | 128| 0.0005 | 0.00001 | ms_euclidean | 0.0355 | 0.1176 |  |
| f02_baseline-15 | 150 | 128| 0.0005 | 0.00001 | reg_mean_euclidean | 0.0425 | 0.1214 | 0.12201 |
| f02_baseline-16 | 400 | 128| 0.0001 | 0.00001 | reg_mean_euclidean | 0.0215 | 0.1279 | 0.1287 |
| f02_baseline-18x | 300 | 128| 0.0005 | 0.00001 | mean_euclidean | 0.0236 | 0.1123 | 0.1116 |

### Errors

In [ ]:
sns.distplot(test_data[FEATURES].as_matrix()[10])

In [ ]:
sns.distplot(scaler.transform(train_data[FEATURES])[10])

In [ ]:
import seaborn as sns
%matplotlib inline


predictions = model_test.predict(
    x={
        'left_imgs':test_imgs_left,
        'right_imgs': test_imgs_right,
        'features': scaler.transform(test_data[FEATURES]),
        #'features': test_data[FEATURES].as_matrix()
    }
)
errors = test_data[TARGETS].as_matrix()-predictions
sns.jointplot(x=errors[:,0], y=errors[:,1], kind="kde")